# Small Caps Invest

## Enregistrement manuel des trades

In [10]:
# Enregistrer un achat ou une vente 

import os
import pandas as pd
from datetime import datetime

csv_file = "transactions.csv"

# Vérifier si le fichier n'existe pas ou s'il est vide pour créer les en-têtes
if not os.path.exists(csv_file) or os.path.getsize(csv_file) == 0:
    headers = "date,ticker,quantite,prix,montant,total_compte,montant_invest\n"
    with open(csv_file, "w") as f:
        f.write(headers)

# demande les détails de la transaction à l'utilisateur
invest = float(input("Entrer un montant investi (ou 0 si aucun) : "))
ticker = input("Entrer le ticker de l'action : ")
quantite = float(int(input("Entrer la quantité achetée (positif) ou vendue (négatif) : ")))
prix = float(input("Entrer le prix unitaire de l'action : "))
date = input("Entrer la date de la transaction (YYYY-MM-DD) : ")
date = datetime.strptime(date, "%Y-%m-%d").date() #convertir en format date
montant = quantite * prix

# Calcul du total_compte et PnL
df = pd.read_csv(csv_file)
if len(df) > 0:
    total_compte = montant + df.iloc[-1]["total_compte"]
    montant_invest = invest + df.iloc[-1]["montant_invest"]
else:
    total_compte = montant
    montant_invest = invest

# Enregistrer l'achat ou la vente dans le fichier CSV   
with open("transactions.csv", "a") as f:
    f.write(f"{date.isoformat()},{ticker},{quantite},{prix},{montant},{total_compte},{montant_invest}\n")

# Afficher le contenu du fichier CSV
df = pd.read_csv("transactions.csv")
print(df.head())


         date ticker  quantite  prix  montant  total_compte  montant_invest
0  2025-06-06   MBOT       6.0   3.4     20.4          20.4           100.0
1  2025-06-06   PESI      11.0   2.3     25.3          45.7           100.0
2  2025-06-08      F       2.0  11.4     22.8          68.5           100.0
3  2025-06-08   MBOT      -3.0   3.1     -9.3          59.2           100.0


In [ ]:
# ou ajout d'une ligne TOTAL

In [ ]:
# mise à jour du portefeuille et du PnL

csv_file = "portfolio.csv"

# Vérifier si le fichier n'existe pas ou s'il est vide pour créer les en-têtes
if not os.path.exists(csv_file) or os.path.getsize(csv_file) == 0:
    headers = "date,ticker,quantite,prix,montant,total_compte,montant_invest\n"
    with open(csv_file, "w") as f:
        f.write(headers)

# avec les données renseignées mettre à jour le portefeuille
# TOTAL des positions
# valeur du portefeuille et PnL

#

# Calculer pour aujourd'hui le récap des positions et le PnL
# Calculer aussi si montant_invest était dans le S&P500





In [ ]:
# Trace le graphique PnL avec comparatif S&P500
import matplotlib.pyplot as plt
import yfinance as yf   
import pandas as pd 

df = pd.read_csv("transactions.csv")    